In [1]:
from pynq import Overlay
import asyncio
from psutil import cpu_percent

ol = Overlay("fact_intrpt.bit")
ol.download()

In [2]:
# IP's addresses
IP_CTRL = 0x00
AP_START = 0x1
GIER = 0x04
IP_IER = 0x08
IP_ISR = 0x0C
INTRPT_AP_DONE = 0x1
INTRPT_AP_READY = 0x2
OUT_REG = 0x10
INP_REG = 0x18

_INTRPT = INTRPT_AP_DONE

fact_ip = ol.fact_intrpt
fact_ip.write(GIER, 0x1)
fact_ip.write(IP_IER, _INTRPT)

In [3]:
# Coroutine that waits for an IP to be done.
async def read_ip(ip):
    while True:
        # Wait for the IP to finish.
        await ip.interrupt.wait()
        # Clear the interrupt and then print output's value.
        if (ip.read(IP_ISR) & _INTRPT):
            ip.write(IP_ISR, _INTRPT)
            print('interrupt received, out = {}'.format(ip.read(OUT_REG)))

# Task for IP using the coroutine
ip_task = asyncio.ensure_future(read_ip(fact_ip))
    
# Coroutine for writing input and starting the IP with delay
async def write_wait(interval):
    await asyncio.sleep(interval)
    # write to input
    fact_ip.write(INP_REG, 10) 
    print("input = ", fact_ip.read(INP_REG))
    fact_ip.write(IP_CTRL, AP_START) # You can comment it out to test the interrupt
    print("IP started")
    await asyncio.sleep(interval)
         
# Run the event loop until the time interval expires
time_interval = 2  # time in seconds
loop = asyncio.get_event_loop()
write_task = asyncio.ensure_future(write_wait(time_interval))

# Using psutil to record CPU utilization.
cpu_percent(percpu=True)  # Initializing the CPU monitoring.
loop.run_until_complete(write_task)
cpu_used = cpu_percent(percpu=True)

# Printing the CPU utilization
print('CPU Utilization = {cpu_used}'.format(**locals()))

# Removing the IP task from the event loop.
ip_task.cancel()

input =  10
IP started
interrupt received, out = 3628800
CPU Utilization = [1.8, 6.5]


True

# IP address map

In [2]:
ol.hierarchy_dict

{}

In [3]:
ol.gpio_dict

{}

In [4]:
ol.ip_dict

{'axi_intc': {'addr_range': 65536,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'axi_intc',
  'gpio': {},
  'interrupts': {},
  'phys_addr': 1098907648,
  'state': None,
  'type': 'xilinx.com:ip:axi_intc:4.1'},
 'fact_intrpt': {'addr_range': 65536,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'fact_intrpt',
  'gpio': {},
  'interrupts': {'interrupt': {'controller': 'axi_intc',
    'fullpath': 'fact_intrpt/interrupt',
    'index': 0}},
  'phys_addr': 1136656384,
  'state': None,
  'type': 'xilinx.com:hls:fact_intrpt:1.0'}}